# Effects em Python - Uma abordagem funcional

Este notebook explora o conceito de **Effects** em programação funcional usando Python. 

Um `Effect` é essencialmente um `thunk` - uma função que não recebe argumentos e retorna um valor. Isso nos permite controlar **quando** e **como** computações são executadas, habilitando composição poderosa de operações.

In [17]:
from typing import Callable

type Effect[A] = Callable[[], A]

In [18]:
import random

# Isso é uma função normal, ela retorna uma int
def random_number() -> int:
    return random.randint(1, 100)

# Essa é uma função que retorna um `Effect` de int
# Ou seja, ela não executa a função imediatamente, mas retorna uma função que, 
# quando chamada, executará a função original e retornará o resultado.
def random_number_function() -> Effect[int]:
  return random_number

# Vamos criar um helper que simplesmente roda o `Effect` e imprime o resultado
def run[A](effect: Effect[A]) -> None:
    print(effect())

run(random_number_function())

62


## Por que usar Effects?

**Porque agora temos uma função que pode ser executada mais tarde!**

Em vez de modificar o resultado, conseguimos modificar a função em si. Por exemplo, podemos repetir a execução de uma função várias vezes.

Note que `repeat` não executa a função imediatamente - ela retorna um novo efeito que retorna uma lista de A.

In [19]:
def repeat[A](effect: Effect[A], times: int) -> Effect[list[A]]:
    def repeated_effect() -> list[A]:
        results: list[A] = []
        for _ in range(times):
            result = effect()
            results.append(result)
        return results

    return repeated_effect

# Agora podemos fazer composição de efeitos
# Podemos repetir a execução de `random_number_function` 5 vezes
run(repeat(random_number_function(), 5))

[18, 75, 30, 72, 50]


Note que até eu fazer `run`, eu não estou executando nada. 
E como repeat retorna um novo `Effect`, eu posso fazer isso várias vezes.

In [20]:
twice = repeat(random_number_function(), 2)
thrice = repeat(random_number_function(), 3)

run(twice)
run(thrice)

[81, 74]
[71, 48, 30]


## Lidando com Erros

O Effect em TypeScript pode lidar com erros elegantemente, mas vamos fazer uma abordagem simples e criar um efeito que pode falhar.

**Lembre-se:** o `thunk` não possui argumentos, mas uma função pode receber argumentos e retornar um `Effect` que usa esses argumentos.

In [21]:
def fail_if_small_number(number: Effect[int]) -> Effect[int]:
    def effect() -> int:
        n = number()
        if n < 50:
            print("Simulating failure for number:", n)
            raise Exception("Random failure")
        return n

    return effect

# Note: preciso definir uma função internamente e retornar ela
# Isso acontece porque Python não é ideal para programação funcional
# e limita lambdas a uma linha
try:
    run(fail_if_small_number(random_number_function()))
except Exception as e:
    print("Error occurred:", e)

73


## Padrão Retry

Novamente, o erro só acontece **DEPOIS** de chamar `run`. 

Vamos criar um efeito que tenta executar um efeito várias vezes até ter sucesso:

In [22]:
def retry[A](effect: Effect[A], times: int) -> Effect[A]:
  def repeated_effect():
      for _ in range(times):
          try:
            return effect()
          except Exception:
            print("Effect failed, retrying...")
            continue
      raise Exception("Effect failed after retries")
  return repeated_effect

# Vamos tentar executar o efeito que falha 3 vezes
run(retry(fail_if_small_number(random_number_function()), 3))

Simulating failure for number: 48
Effect failed, retrying...
Simulating failure for number: 14
Effect failed, retrying...
87


## Padrão OrElse

Puta merda, eu fico meio puto quando falha! 😅

Existe um padrão comum que é o `orElse` - se falhar algo ali dentro, eu quero retornar um valor padrão.

**SEMPRE, SEMPRE** retornamos um `Effect`. Isso permite que todas as funções sejam compostas e encadeadas.

In [23]:
def or_else[A](effect: Effect[A], fallback: A) -> Effect[A]:
    def wrapped_effect() -> A:
        try:
            return effect()
        except Exception:
            print("Effect failed, returning fallback value:", fallback)
            return fallback
    return wrapped_effect

# Se falhar mesmo depois de tentar, vira 0
run(repeat(or_else(retry(fail_if_small_number(random_number_function()), 2), 0), 20))

Simulating failure for number: 2
Effect failed, retrying...
Simulating failure for number: 48
Effect failed, retrying...
Simulating failure for number: 28
Effect failed, retrying...
Effect failed, returning fallback value: 0
Simulating failure for number: 44
Effect failed, retrying...
Simulating failure for number: 24
Effect failed, retrying...
Simulating failure for number: 9
Effect failed, retrying...
Effect failed, returning fallback value: 0
Simulating failure for number: 45
Effect failed, retrying...
Simulating failure for number: 39
Effect failed, retrying...
Simulating failure for number: 27
Effect failed, retrying...
Effect failed, returning fallback value: 0
Simulating failure for number: 34
Effect failed, retrying...
Simulating failure for number: 9
Effect failed, retrying...
Simulating failure for number: 35
Effect failed, retrying...
Effect failed, returning fallback value: 0
Simulating failure for number: 1
Effect failed, retrying...
Simulating failure for number: 23
Effec

## Telemetria e Observabilidade

O legal é que como temos controle de **QUANDO** executar o efeito, podemos adicionar funções antes ou depois dele:

In [24]:
def telemetry[A](effect: Effect[A], text: str) -> Effect[A]:
    def wrapped_effect() -> A:
        print(f"Starting effect: {text}")
        result = effect()
        print(f"Effect completed with result: {result}")
        return result
    return wrapped_effect

run(telemetry(repeat(or_else(fail_if_small_number(random_number_function()), 0), 10), "Repeat Effect"))

Starting effect: Repeat Effect
Simulating failure for number: 34
Effect failed, returning fallback value: 0
Simulating failure for number: 35
Effect failed, returning fallback value: 0
Simulating failure for number: 42
Effect failed, returning fallback value: 0
Simulating failure for number: 12
Effect failed, returning fallback value: 0
Simulating failure for number: 32
Effect failed, returning fallback value: 0
Simulating failure for number: 1
Effect failed, returning fallback value: 0
Effect completed with result: [0, 89, 0, 0, 0, 0, 97, 62, 0, 82]
[0, 89, 0, 0, 0, 0, 97, 62, 0, 82]


O Effect não precisa sempre retornar A, ele pode retornar outros valores
Por exemplo, aqui a gente adiciona o resultado do resulto MAIS quanto tempo demorou

In [25]:
from typing import Tuple
from time import perf_counter

def timed[A](effect: Effect[A]) -> Effect[Tuple[A, float]]:
    def timed_effect() -> Tuple[A, float]:
        start = perf_counter()
        result = effect()
        end = perf_counter()
        return (result, (end - start) * 10000)

    return timed_effect

# Vamos medir o tempo de execução do efeito
run(timed(repeat(or_else(retry(fail_if_small_number(telemetry(random_number_function(), "Random number generator"),), 2), 0), 20)))

Starting effect: Random number generator
Effect completed with result: 44
Simulating failure for number: 44
Effect failed, retrying...
Starting effect: Random number generator
Effect completed with result: 28
Simulating failure for number: 28
Effect failed, retrying...
Effect failed, returning fallback value: 0
Starting effect: Random number generator
Effect completed with result: 67
Starting effect: Random number generator
Effect completed with result: 56
Starting effect: Random number generator
Effect completed with result: 72
Starting effect: Random number generator
Effect completed with result: 94
Starting effect: Random number generator
Effect completed with result: 68
Starting effect: Random number generator
Effect completed with result: 49
Simulating failure for number: 49
Effect failed, retrying...
Starting effect: Random number generator
Effect completed with result: 95
Starting effect: Random number generator
Effect completed with result: 43
Simulating failure for number: 43


## O Problema dos Parênteses Aninhados

Vai tomar no cu, tem tanto parêntesis que eu fiquei perdido! 😅

As funções estão todas encadeadas e eu não consigo ver o que está acontecendo. Preciso ler de dentro para fora.

**Solução:** No UNIX a solução é encadear comandos com pipe!
- Exemplo: `ls -l | grep py | wc -l` em vez de `wc -l (grep py (ls -l))`
- O `toolz` tem uma função chamada `pipe` que faz isso!

In [26]:
from toolz import pipe # type: ignore

# O pipe faz o seguinte:
# pipe(data, f, g, h) === h(g(f(data)))

# Nossas funções recebem um `Effect` como primeiro argumento
# portanto, precisamos criar uma função inline que recebe o `Effect` e retorna o resultado
# isso é o lambda - ele passa uma função que age em cima do resultado do `Effect`

result = pipe( # type: ignore
    random_number_function(),# O primeiro valor do pipe é um valor de verdade  # type: ignore
    lambda effect: telemetry(effect, "Random number generator"), # type: ignore
    fail_if_small_number, # Nesse caso, o primeiro argumento é o Effect, então funciona 
    lambda effect: retry(effect, 2), # type: ignore
    lambda effect: or_else(effect, 0), # type: ignore
    lambda effect: repeat(effect, 20), # type: ignore
    timed, # como não precisa de outros argumentos, podemos passar diretamente
    run # o mesmo para o run
)

# Agora eu consigo ler de fora para dentro
# E consigo ver o que está acontecendo em cada etapa
# Como se fosse um pipeline de dados

Starting effect: Random number generator
Effect completed with result: 54
Starting effect: Random number generator
Effect completed with result: 71
Starting effect: Random number generator
Effect completed with result: 9
Simulating failure for number: 9
Effect failed, retrying...
Starting effect: Random number generator
Effect completed with result: 7
Simulating failure for number: 7
Effect failed, retrying...
Effect failed, returning fallback value: 0
Starting effect: Random number generator
Effect completed with result: 97
Starting effect: Random number generator
Effect completed with result: 36
Simulating failure for number: 36
Effect failed, retrying...
Starting effect: Random number generator
Effect completed with result: 70
Starting effect: Random number generator
Effect completed with result: 77
Starting effect: Random number generator
Effect completed with result: 49
Simulating failure for number: 49
Effect failed, retrying...
Starting effect: Random number generator
Effect com

## Partial Application - Fazendo Melhor Ainda

Ainda conseguimos fazer melhor!

O `pipe` espera uma função que recebe somente um argumento, mas nossas funções recebem um `Effect` como primeiro argumento e às vezes passam outro argumento.

**Solução:** Técnica chamada `partial application` - criar uma função que recebe alguns argumentos e retorna uma nova função que recebe o restante dos argumentos.

Como: `sum(a, b) === partial_sum(a)(b)`

In [27]:
from toolz import partial # type: ignore

# partial retorna uma função que recebe menos argumentos do que a função original
# e quando essa função é chamada, ela chama a função original com os argumentos restantes.

def with_repeat[A](times: int) -> Callable[[Effect[A]], Effect[list[A]]]:
  return partial(repeat, times=times) # type: ignore

# with_repeat(5)(random_number_function()) == repeat(random_number_function(), 5)

def with_retry[A](times: int) -> Callable[[Effect[A]], Effect[A]]:
    return partial(retry, times=times) # type: ignore

def with_or_else[A](fallback: A) -> Callable[[Effect[A]], Effect[A]]:
    return partial(or_else, fallback=fallback) # type: ignore

def with_telemetry[A](text: str) -> Callable[[Effect[A]], Effect[A]]:
    return partial(telemetry, text=text) # type: ignore


pipe( # type: ignore
    random_number_function(), 
    with_telemetry("Random number generator"), 
    fail_if_small_number, 
    with_retry(2), 
    with_or_else(0), 
    with_repeat(20), 
    timed, 
    run
)

# Você pode ler pensando:
# Rodar esse efeito com telemetria,
# se falhar, tentar mais duas vezes,
# se falhar, retornar 0,
# repetir 20 vezes,
# medir o tempo de execução

Starting effect: Random number generator
Effect completed with result: 22
Simulating failure for number: 22
Effect failed, retrying...
Starting effect: Random number generator
Effect completed with result: 76
Starting effect: Random number generator
Effect completed with result: 6
Simulating failure for number: 6
Effect failed, retrying...
Starting effect: Random number generator
Effect completed with result: 88
Starting effect: Random number generator
Effect completed with result: 73
Starting effect: Random number generator
Effect completed with result: 3
Simulating failure for number: 3
Effect failed, retrying...
Starting effect: Random number generator
Effect completed with result: 77
Starting effect: Random number generator
Effect completed with result: 47
Simulating failure for number: 47
Effect failed, retrying...
Starting effect: Random number generator
Effect completed with result: 77
Starting effect: Random number generator
Effect completed with result: 73
Starting effect: Ran

## Map e FlatMap - Transformações Poderosas

Duas funções muito úteis:

### Map
É **LITERALMENTE** a mesma coisa que um map de list. Pega o resultado do efeito e aplica uma função.

### FlatMap 
Mas e se a função que eu quero aplicar também for um efeito? Nesse caso, a gente "amassa" os dois - geralmente chamam isso de `flat_map`.

In [28]:
def map_effect[A, B](effect: Effect[A], func: Callable[[A], B]) -> Effect[B]:
    def mapped_effect() -> B:
        result = effect()
        return func(result)
    return mapped_effect 

# Isso permite usar lambdas diretamente
run(map_effect(random_number_function(), lambda x: f"Número aleatório: {x}"))

def flat_map_effect[A, B](effect: Effect[A], func: Callable[[A], Effect[B]]) -> Effect[B]:
    return func(effect())  # Chama o efeito original, depois chama o efeito que a função recebeu

def duplicate_effect(effect: Effect[int]) -> Effect[int]:
    def duplicated_effect() -> int:
        return effect() * 2
    return duplicated_effect

run(flat_map_effect(random_number_function, duplicate_effect))

Número aleatório: 17
154


## Generators para Simplificar

Uma coisa chata é a necessidade de definir uma função para cada coisa. 

Vamos usar **generators** para tornar isso mais elegante! O decorador `@gen` permite escrever código que parece síncrono mas funciona com Effects.
Ignorando types porque eu não sou obrigado a aprender e eu acho que 
o sistema de tipos do Python não tanka tanta loucura


In [29]:
# type: ignore
def gen(generator_func):
    def run_effect():
        generator = generator_func()
        
        try:
            # Start the generator and get the first yielded Effect
            current_effect = next(generator)
            
            while True:
                # Run the yielded Effect to get its result
                result = current_effect()
                
                try:
                    # Send the result back to the generator and get the next Effect
                    current_effect = generator.send(result)
                except StopIteration as e:
                    # Generator completed, return its final value
                    return e.value
                    
        except StopIteration as e:
            # Generator completed without yielding anything
            return e.value
    
    return run_effect

@gen
def my_effect():
    a = yield random_number_function()
    b = yield random_number_function()

    if a > b:
        return f"Primeiro número era maior: {a}"
    else:
        return f"Segundo numero era maior: {b}"

# Ainda é um efeito! Então podemos fazer
run(repeat(my_effect, 2))

['Primeiro número era maior: 25', 'Primeiro número era maior: 41']


## Exemplo Real - Sistema de Notificações

Vamos fazer um exemplo real! Primeiro, algumas funções e utilitários:

1. Efeito para buscar dados
2. Telemetria
3. Logs de debug e erro
4. Enviar para zap e email
5. Tentar novamente se o efeito retornar um exception


In [ ]:
from typing import TypedDict, Optional
from time import sleep
from termcolor import colored

emails = [
    "john.doe@example.com",
    "jane.smith@example.com",
    "alice.johnson@example.com",
    "bob.brown@example.com",
    "charlie.davis@example.com",
    "emily.clark@example.com",
    "frank.harris@example.com",
    "grace.lee@example.com",
    "henry.miller@example.com",
    "ivy.wilson@example.com",
]

class User(TypedDict):
    id: int
    whatsapp: Optional[str]
    email: str

def get_user(id: int) -> Effect[User | Exception]:
    # 50% de chance de dar erro na API
    def wrapped_effect():
        if random.random() < 0.5:
            return Exception("Erro ao buscar usuário")
        return User({
            "id": id,
            "whatsapp": random.choice([None, "1234-5678"]),
            "email": emails[id-1]
        })
    
    return wrapped_effect

def retry_if_exception[A](effect: Effect[A], retries: int) -> Effect[A | Exception]:
    def wrapped_effect():
        for _ in range(retries):
            result = effect()
            if not isinstance(result, Exception):
                return result
        return Exception("Max retries exceeded")
    return wrapped_effect

def send_zap_zap(to: str, message: str) -> Effect[str | Exception]:
    def wrapped_effect():
        sleep(random.random())
        if random.random() < 0.5:
            return Exception("Erro ao enviar mensagem pelo WhatsApp")
        return(f"Zap enviada para {to}")
    return wrapped_effect

def send_email(to: str, subject: str) -> Effect[str | Exception]:
    def wrapped_effect():
        sleep(random.random())
        if random.random() < 0.5:
            return Exception("Erro ao enviar email")
        return(f"Email enviado para {to}")
    return wrapped_effect

def debug_to_gcp[A](effect: Effect[A], message: str) -> Effect[A]: 
    def wrapped_effect():
        result = effect()
        print(colored(f"DEBUG: {message}", 'green'))
        return result
    return wrapped_effect

def log_error[A](effect: Effect[A]) -> Effect[A]:
    def wrapped_effect():
        result = effect()
        if isinstance(result, Exception):
            print(colored(f"ERROR: {result}", "red"))
        return result
    return wrapped_effect

def telemetry_effect[A](effect: Effect[A], message: str) -> Effect[A]:
    def wrapped_effect() -> A:
        timed_effect = timed(effect)

        def time_message(pair: tuple[A, float]) -> Effect[A]:
            msg = f"{message}: {pair[1]/10000:.2f} seconds"
            return debug_to_gcp(lambda: pair[0], msg)

        logged = flat_map_effect(timed_effect, time_message)
        err_logged = log_error(logged)
        return err_logged()
    return wrapped_effect


def zap_user(number: str, message: str) -> Effect[str | Exception]:
    return telemetry_effect(send_zap_zap(number, message), f"zap_user: {number}")

def email_user(email: str, subject: str) -> Effect[str | Exception]:
    return telemetry_effect(send_email(email, subject), f"email_user: {email}")


def run_all_effects[A](effects: list[Effect[A]]) -> None:
  results: list[A] = []
  for effect in effects:
      result = effect()
      results.append(result)

  print(results)

## Escrevendo nossa main

Não só é fácil fazer composição, mas com o `gen` você
consegue escrever de forma mais "imperativa",
incluindo os `if` dá vida. 

Agora força o olho e imagina que `yield` na verdade é `await` - **é a mesma coisa!**

Com generators podemos escrever código que parece síncrono mas mantém todas as vantagens dos Effects:

In [45]:
# type: ignore

def main(id: int):
      
  @gen
  def generator():
      user = yield retry_if_exception(get_user(id), 4)

      # Aqui podemos utilizar um fluxo normal
      if isinstance(user, Exception):
          return f'Error ao buscar usuário {id}'

      if user["whatsapp"]:
          response = yield retry_if_exception(zap_user(user["whatsapp"], "Olá, você tem uma nova mensagem!"), 2)

      elif user["email"]:
          response = yield retry_if_exception(email_user(user["email"], "Nova mensagem recebida"), 2)

      if isinstance(response, Exception):
          return f"Erro ao enviar mensagem para usuário id {user['id']}"
      
      return response

  return generator


run_all_effects([
  main(1)
])

DEBUG: zap_user: 1234-5678: 0.71 seconds
ERROR: Erro ao enviar mensagem pelo WhatsApp
DEBUG: zap_user: 1234-5678: 0.49 seconds
Zap enviada para 1234-5678
DEBUG: zap_user: 1234-5678: 0.49 seconds
Zap enviada para 1234-5678
